# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 9:29AM,256591,10,Eme-108438,Emerginnova,Released
1,Feb 3 2023 9:29AM,256591,10,Enova-11310,Emerginnova,Released
2,Feb 3 2023 9:29AM,256591,10,Enova-11311,Emerginnova,Released
3,Feb 3 2023 9:23AM,256589,19,ADV80011310,"AdvaGen Pharma, Ltd",Released
4,Feb 3 2023 9:23AM,256589,19,ADV80011323,"AdvaGen Pharma, Ltd",Released
5,Feb 3 2023 9:23AM,256589,19,ADV80011324,"AdvaGen Pharma, Ltd",Released
6,Feb 3 2023 9:23AM,256589,19,ADV80011337,"AdvaGen Pharma, Ltd",Released
7,Feb 3 2023 9:18AM,256588,10,0086352265,ISDIN Corporation,Released
8,Feb 3 2023 9:13AM,256587,10,9111283,"Citieffe, Inc.",Released
9,Feb 3 2023 9:10AM,256585,10,0086353208,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,256586,Released,1
29,256587,Released,1
30,256588,Released,1
31,256589,Released,4
32,256591,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256586,NaN,NaN,1.0
256587,NaN,NaN,1.0
256588,NaN,NaN,1.0
256589,NaN,NaN,4.0
256591,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256471,10.0,0.0,3.0
256476,0.0,1.0,0.0
256482,0.0,0.0,1.0
256484,0.0,0.0,1.0
256487,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256471,10,0,3
256476,0,1,0
256482,0,0,1
256484,0,0,1
256487,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256471,10,0,3
1,256476,0,1,0
2,256482,0,0,1
3,256484,0,0,1
4,256487,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256471,10,,3
1,256476,,1,
2,256482,,,1
3,256484,,,1
4,256487,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 9:29AM,256591,10,Emerginnova
3,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd"
7,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation
8,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc."
9,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation
31,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions
32,Feb 3 2023 9:02AM,256584,19,"AdvaGen Pharma, Ltd"
33,Feb 3 2023 8:57AM,256581,12,Hush Hush
48,Feb 3 2023 8:36AM,256583,19,"AdvaGen Pharma, Ltd"
63,Feb 3 2023 8:30AM,256582,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 3 2023 9:29AM,256591,10,Emerginnova,,,3
1,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",,,4
2,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,,,1
3,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc.",,,1
4,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,,,22
5,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions,,,1
6,Feb 3 2023 9:02AM,256584,19,"AdvaGen Pharma, Ltd",,,1
7,Feb 3 2023 8:57AM,256581,12,Hush Hush,,,15
8,Feb 3 2023 8:36AM,256583,19,"AdvaGen Pharma, Ltd",,2,13
9,Feb 3 2023 8:30AM,256582,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:29AM,256591,10,Emerginnova,3,,
1,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",4,,
2,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,1,,
3,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc.",1,,
4,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,22,,
5,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions,1,,
6,Feb 3 2023 9:02AM,256584,19,"AdvaGen Pharma, Ltd",1,,
7,Feb 3 2023 8:57AM,256581,12,Hush Hush,15,,
8,Feb 3 2023 8:36AM,256583,19,"AdvaGen Pharma, Ltd",13,2,
9,Feb 3 2023 8:30AM,256582,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:29AM,256591,10,Emerginnova,3,,
1,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",4,,
2,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,1,,
3,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc.",1,,
4,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,22,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:29AM,256591,10,Emerginnova,3.0,NaN,NaN
1,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",4.0,NaN,NaN
2,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,1.0,NaN,NaN
3,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,22.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:29AM,256591,10,Emerginnova,3.0,0.0,0.0
1,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",4.0,0.0,0.0
2,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,1.0,0.0,0.0
3,Feb 3 2023 9:13AM,256587,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,22.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2052502,36.0,1.0,0.0
12,1026191,16.0,2.0,0.0
15,256471,3.0,0.0,10.0
16,769665,2.0,1.0,0.0
19,1539378,20.0,7.0,0.0
21,1795625,5.0,2.0,0.0
22,513067,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2052502,36.0,1.0,0.0
1,12,1026191,16.0,2.0,0.0
2,15,256471,3.0,0.0,10.0
3,16,769665,2.0,1.0,0.0
4,19,1539378,20.0,7.0,0.0
5,21,1795625,5.0,2.0,0.0
6,22,513067,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,1.0,0.0
1,12,16.0,2.0,0.0
2,15,3.0,0.0,10.0
3,16,2.0,1.0,0.0
4,19,20.0,7.0,0.0
5,21,5.0,2.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,12,Released,16.0
2,15,Released,3.0
3,16,Released,2.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
Executing,1.0,2.0,0.0,1.0,7.0,2.0,0.0
Released,36.0,16.0,3.0,2.0,20.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,0.0,1.0,7.0,2.0,0.0
2,Released,36.0,16.0,3.0,2.0,20.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,0.0,1.0,7.0,2.0,0.0
2,Released,36.0,16.0,3.0,2.0,20.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()